In [1]:
import regex as re
from functional import seq
from fn import _
from collections import namedtuple
from typing import List, Dict, Tuple, Optional
from termcolor import colored
import os

In [2]:
#Nr doesnt matter, position does

In [3]:
act_dir = os.environ["HOME"]+"/tmp/nlp/ustawy"
#act_file = "1993_599.txt"
act_file = "2003_592.txt"

In [4]:
act = open(act_dir + "/" + act_file).read()
print(act[:1000])




Tekst ustawy
ustalony ostatecznie po rozpatrzeniu poprawek Senatu
 
 
 
USTAWA
z dnia 5 września
2008 r.
 
o komercjalizacji
państwowego przedsiębiorstwa użyteczności publicznej "Poczta Polska" 
 

Rozdział 1 
Przepisy ogólne 
 
Art. 1.
Ustawa reguluje
zasady i tryb komercjalizacji państwowego przedsiębiorstwa użyteczności
publicznej "Poczta Polska" działającego na podstawie ustawy z dnia 30 lipca
1997 r. o państwowym przedsiębiorstwie użyteczności publicznej "Poczta Polska"
(Dz. U. Nr 106, poz. 675, z późn. zm.[1])),
zwanego dalej "Pocztą Polską". 
 
Art. 2.
1.  Komercjalizacja
Poczty Polskiej, w rozumieniu ustawy, polega na przekształceniu Poczty Polskiej
w spółkę akcyjną o nazwie "Poczta Polska Spółka Akcyjna", zwaną dalej "Spółką",
w której Skarb Państwa jest jedynym akcjonariuszem. 
2.  Spółka
wstępuje we wszystkie stosunki prawne, których podmiotem była Poczta Polska,
bez względu na charakter prawny tych stosunków, jeżeli przepisy ustawy nie
stanowią inaczej. 
 
Art. 3.
Spółce

In [5]:
def match_with_index(pattern ,text:str)-> List[Tuple[int,int,str]]:
    return [(m.start(0), m.end(0),m.captures()[0]) for m in re.finditer(pattern, text)]


def print_highlighted(
    text:str,
    matches_groups:List[List[Tuple[int,int]]],
    colors:List[Tuple[str,Optional[str]]]
):
    if len(matches_groups) != len(colors):
        raise "There should be the same number of matches groups and colors"
    
    MatchedEntry=namedtuple('MatchedEntry','beg end color on_color')
    
    # Seq[MatchedEntry]
    mg = seq(matches_groups)\
        .zip(colors)\
        .flat_map(lambda gc: seq(gc[0])\
            .map(lambda x : MatchedEntry(beg=x[0],end=x[1],color=gc[1][0], on_color = gc[1][1]))
                 )\
        .order_by(_.beg)
        
    # Will duplicate some matches but that's not the point
    beg = 0
    
    for matched in mg:
        print(text[beg:matched.beg],end="") 
        print(colored(text[matched.beg:matched.end],color= matched.color, on_color= matched.on_color),end="")
        beg = matched.end

            
        
        
    
                
    

In [17]:
dzu_pat = re.compile("\Dz.U.")
year_pat = re.compile("((199)|(198)|(200)|(201))\d{1}")
nr_pat = re.compile("Nr\s+\d+") #TODO: word sep or sth
pos_pat = re.compile("poz\.\s+\d+(-\d+)*") #todo - "i \d"
ustaw_pat=re.compile("(U|u)staw\p{Ll}*\s+z\s+dnia\s+\d+\s+\w+") #todo:bcm - and year
# todo: oraz, commas, i 
#TODO: pos

In [18]:
matches = seq([
    year_pat,
    dzu_pat,
    nr_pat,
    pos_pat,
    ustaw_pat
]).map(lambda pattern: match_with_index(pattern,act)).to_list()

print_highlighted(act,matches
                  ,[
                      ("green","on_grey"),
                      ("red","on_grey"),
                      ("blue","on_grey"),
                      ("yellow","on_grey"),
                      ("white","on_grey"),
                  ])




Tekst ustawy
ustalony ostatecznie po rozpatrzeniu poprawek Senatu
 
 
 
USTAWA
z dnia 5 września
2008 r.
 
o komercjalizacji
państwowego przedsiębiorstwa użyteczności publicznej "Poczta Polska" 
 

Rozdział 1 
Przepisy ogólne 
 
Art. 1.
Ustawa reguluje
zasady i tryb komercjalizacji państwowego przedsiębiorstwa użyteczności
publicznej "Poczta Polska" działającego na podstawie ustawy z dnia 30 lipca
1997 r. o państwowym przedsiębiorstwie użyteczności publicznej "Poczta Polska"
(Dz. U. Nr 106, poz. 675, z późn. zm.[1])),
zwanego dalej "Pocztą Polską". 
 
Art. 2.
1.  Komercjalizacja
Poczty Polskiej, w rozumieniu ustawy, polega na przekształceniu Poczty Polskiej
w spółkę akcyjną o nazwie "Poczta Polska Spółka Akcyjna", zwaną dalej "Spółką",
w której Skarb Państwa jest jedynym akcjonariuszem. 
2.  Spółka
wstępuje we wszystkie stosunki prawne, których podmiotem była Poczta Polska,
bez względu na charakter prawny tych stosunków, jeżeli przepisy ustawy nie
stanowią inaczej. 
 
Art. 3.
Spółce

Find all external references to bills, e.g. **ustawie z dnia 4 marca 1994 r. o zakładowym funduszu świadczeń socjalnych (Dz. U. z 2012 r. poz. 592)**. The result should be aggregated by bill ID (year and position) and sorted by descending number of reference counts. The reference format should include:
        the title of the regulation (if present)
        the year of the regulation
        the number of the Journal of Laws of the Republic of Poland (Dziennik Ustaw) - if applicable
        the position of the regulation
